In [3]:
# Part 1
import re
import itertools

data = open("input_12.txt").read().splitlines()
data = [x.split() for x in data]
print(data)
# [print(line) for line in data]

total_combos = 0
# for each line in data:
for i,line in enumerate(data):
    key = [int(x) for x in line[1].split(',')]
    # Set up the line[0]ing for searching
    line[0] = line[0].join(['.','.']) # add '.' at the ends to help search line[0]ings later
    unknown = [i for i, letter in enumerate(line[0]) if letter == '?']
    broken = [x.start(0) for x in re.finditer(r'#',line[0])]
    # print(broken,unknown)
    test_combos = list(itertools.combinations(unknown, sum(key)-len(broken)))
    # print(test_combos)
    line[0] = re.sub(r'\?',r'.',line[0])  # fill in the ? with .
    # print(line[0],'key =',key,'broken =',broken,'unknown =',unknown)
    regexstr = ''
    for k in range(len(key)):
        regexstr = regexstr + r'\.+#{' + str(key[k]) + r'}'
    # print('regex = ',regexstr)

    # loop over the possible combinations of broken springs and validate
    for k,combo in enumerate(test_combos):
        temp = line[0]
        # temp[combo] = ''
        temp = [ch for ch in temp]
        for m in range(len(combo)):
            temp[combo[m]] = '#'
        temp = ''.join(temp)
        
        z = re.findall(regexstr,temp)
        if z!=[]:
            total_combos += 1
            # print(temp,z,'valid =',z!=[])

    # print(' ')
    # print('       end of line')
    # print(' ')

print('total combos = ',total_combos)


[['????.??#.?????#?', '1,1,1,1,6'], ['??.####?.??#?#?.?.??', '4,6'], ['????????#?..?', '2,7,1'], ['??.##????????', '2,3,4'], ['.####??#???#.????.?', '11,4,1'], ['???..????.????#??', '1,1,1,1,5'], ['#???.#??#??#????#', '4,1,5,1,1'], ['?...???##?????#??', '1,12'], ['?.???.?????????', '1,1,1,2,1'], ['???.?#.#??#???.????', '3,1,1,3,1,1'], ['???.??????.???#', '1,5,1,1'], ['??#?.#??.??', '2,1,1'], ['?#?.??#?###?.?.?#.#', '1,3,4,1,2,1'], ['???????#??????#?.', '9,6'], ['?#???#???#??', '2,2,3'], ['????.????????', '3,2,4'], ['..????#?.??', '1,4,1'], ['.???????#?', '1,4'], ['??#???????#?.', '1,1,1,5'], ['?.??#?????????#?', '1,13'], ['????.?#???..', '1,1,1,1'], ['#.?#?##???#??.????', '1,9,4'], ['.???#???.??##?', '3,3'], ['???###?##??..?.?', '9,1'], ['#?.?#?????????##????', '1,13'], ['???#.?????#???.??.?', '4,1,7,1,1'], ['??????#??.??.?', '2,1,1'], ['.????#??????', '7,2'], ['??????#..????', '1,2,1,1'], ['?.??#.#.#?##?', '1,1,1,4'], ['.??.#??##??#.#????', '1,1,6,1,1'], ['??#??#???????#??.', '8,2'], 

In [17]:
# Part 2
import re
import itertools

# count combos in 1 line
def combo_count(regexstr,line,key):

    # Set up the line for searching
    line = line.join(['.','.']) # add '.' at the ends to help search line[0]ings later
    unknown = [i for i, letter in enumerate(line) if letter == '?']
    broken = [x.start(0) for x in re.finditer(r'#',line)]
    print(broken,unknown)
    test_combos = list(itertools.combinations(unknown, sum(key)-len(broken)))
    print(test_combos)
    line[0] = re.sub(r'\?',r'.',line[0])  # fill in the ? with .

    # look thru the possible combinations and validate them
    combos = 0
    for k,combo in enumerate(test_combos):
        temp = line
        # temp[combo] = ''
        temp = [ch for ch in temp]
        for m in range(len(combo)):
            temp[combo[m]] = '#'
        temp = ''.join(temp)
        
        z = re.findall(regexstr,temp)
        if z!=[]:
            combos += 1
        # print(temp,z,'valid =',z!=[])
    return combos

# main
data = open("input_12_test.txt").read().splitlines()
data = [x.split() for x in data]

# data = [data[x] for x in [0,2,3]]  # cut down the list for testing
print(data)
[print(line) for line in data]

total_combos = 0
# for each line in data:
for i,line in enumerate(data):
    key = [int(x) for x in line[1].split(',')]

    # unfold the lines
    # line[0] = line[0]+'?'+line[0]
    # key = 2*key
    print(line[0],key)

    # smarter way
    if line[0][-2] == '#':
        line2 = line[0]
    else:
        line2 = '?'+line[0]
    print(line2)
    print(' ')

    # print(line[0],'key =',key,'broken =',broken,'unknown =',unknown)
    regexstr = ''
    for k in range(len(key)):
        regexstr = regexstr + r'\.+#{' + str(key[k]) + r'}'
    # print('regex = ',regexstr)

    # loop over the possible combinations of broken springs and validate
    combos = combo_count(regexstr,line[0],key)
    print('                               combos =',combos)
    total_combos += combos

    combos = combo_count(regexstr,line2,key)
    print('                               combos =',combos)
    total_combos += combos

    # print(' ')
    # print('       end of line')
    # print(' ')

print('total combos = ',total_combos)


[['???.###', '1,1,3'], ['.??..??...?##.', '1,1,3'], ['?#?#?#?#?#?#?#?', '1,3,1,6'], ['????.#...#...', '4,1,1'], ['????.######..#####.', '1,6,5'], ['?###????????', '3,2,1']]
['???.###', '1,1,3']
['.??..??...?##.', '1,1,3']
['?#?#?#?#?#?#?#?', '1,3,1,6']
['????.#...#...', '4,1,1']
['????.######..#####.', '1,6,5']
['?###????????', '3,2,1']
???.### [1, 1, 3]
???.###
 
[5, 6, 7] []
[]
                               combos = 0
[5, 6, 7] []
[]
                               combos = 0
.??..??...?##. [1, 1, 3]
.??..??...?##.
 
[12, 13] []
[]
                               combos = 0
[12, 13] []
[]
                               combos = 0
?#?#?#?#?#?#?#? [1, 3, 1, 6]
?#?#?#?#?#?#?#?
 
[2, 4, 6, 8, 10, 12, 14] []
[]
                               combos = 0
[2, 4, 6, 8, 10, 12, 14] []
[]
                               combos = 0
????.#...#... [4, 1, 1]
?????.#...#...
 
[6, 10] []
[]
                               combos = 0
[6, 10] []
[]
                               combos = 0
????.######..##

In [6]:
# sandbox
4**6
16384//2

4**7
test_combos = list(itertools.combinations([1,2,3,4,5],4))
print(len(test_combos))
5**4

a = [1,2,3,4,5]
b = [a[x] for x in [0,2,3]]
print(b)
test_combos = list(itertools.combinations([1,2,3],2))
print(len(test_combos))
4*8*8*8*8



5
[1, 3, 4]
3


16384